In [53]:
import numpy as np
import pandas as pd
import math
import csv
import jieba

In [54]:
csv_file=open('train.csv')    #打开csv文件
test_file=open('test_without_label.csv')
csv_reader_lines = csv.reader(csv_file)  #逐行读取csv文件
test_reader_lines = csv.reader(test_file)
csv_data=[]
test_data=[]
for line in csv_reader_lines:
    csv_data.append("".join(line))
for line in test_reader_lines:
    test_data.append("".join(line))

In [55]:
csv_data[0]

'title tmovie_companies mcmovie_info mi#t.id=mc.movie_idt.id=mi.movie_id#t.production_year<1968mi.info_type_id>15#4094363'

In [56]:
jieba.load_userdict("dict.txt")
interpunctuations=['#','##',' ','title','t','movie_companies','mc','cast_info','ci','movie_info','mi','movie_info_idx','mi_idx','movie_keyword','mk']
words=jieba.lcut(csv_data[0])
cutwords2 = [word for word in words if word not in interpunctuations]   #去除标点符号
cutwords2[:-1]

['t.id',
 '=',
 'mc.movie_id',
 't.id',
 '=',
 'mi.movie_id',
 't.production_year',
 '<',
 '1968',
 'mi.info_type_id',
 '>',
 '15']

In [57]:
features=[]
test_features = []
size=[]
for line in csv_data:
    words=jieba.lcut(line)
    cutwords = [word for word in words if word not in interpunctuations]   #去除标点符号
    features.append(cutwords[:-1])
    size.append(float(cutwords[-1]))
for line in test_data:
    words=jieba.lcut(line)
    cutwords = [word for word in words if word not in interpunctuations]   #去除标点符号
    test_features.append(cutwords)

In [58]:
df = pd.DataFrame()
test_df = pd.DataFrame()

df['feature']=features
df['size']=size
test_df['feature'] = test_features

In [59]:
df.to_csv("features1.csv")
test_df.to_csv("test_feature.csv")

In [60]:
column=['t.id','t.kind_id','t.production_year','mc.id','mc.company_id','mc.movie_id','mc.company_type_id','ci.id','ci.movie_id','ci.person_id'
,'ci.role_id','mi.id','mi.movie_id','mi.info_type_id','mi_idx.id','mi_idx.movie_id','mi_idx.info_type_id','mk.id','mk.movie_id','mk.keyword_id']
min = [1,1,1880,1,1,2,1,1,1,1,1,1,1,1,1,2,99,1,2,1]
max = [2528312,7,2019,2609129,234997,2525745,2,36244344,2525975,4061926,11,14835720,2526430,110,1380035,2525793,113,4523930,2525971,134170]
upper=np.zeros(20)
lower=np.zeros(20)
link=np.zeros(20)

test_upper = np.zeros(20)
test_lower = np.zeros(20)
test_link = np.zeros(20)

In [61]:
def to_vec(cutwords):
    iter=0
    while(iter<len(cutwords)):
        id1=column.index(cutwords[iter])
        link[id1]=1;
        if(cutwords[iter+1]=='>'):
            lower[id1]=(float(cutwords[iter+2])-min[id1])/(max[id1]-min[id1])
        elif(cutwords[iter+1]=='<'):
            upper[id1]=(float(cutwords[iter+2])-min[id1])/(max[id1]-min[id1])
        elif(cutwords[iter+1]=='='):
            word=cutwords[iter+2]
            if word in column:
                id2=column.index(word)
                link[id2]=1.0
            else :
                number=(float(word)-min[id1])/(max[id1]-min[id1])
                upper[id1]=number
                lower[id1]=number
        iter=iter+3

In [62]:
to_vec(cutwords2[:-1])
link

array([1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0.])

In [63]:
uppers=[]
lowers=[]
links=[]
for line in features:
    upper=np.ones(20)
    lower=np.zeros(20)
    link=np.zeros(20)
    to_vec(line)
    uppers.append(upper)
    lowers.append(lower)
    links.append(link)

In [64]:
test_uppers=[]
test_lowers=[]
test_links=[]
for line in test_features:
    upper=np.ones(20)
    lower=np.zeros(20)
    link=np.zeros(20)
    to_vec(line)
    test_uppers.append(upper)
    test_lowers.append(lower)
    test_links.append(link)

In [65]:
df['upper']=uppers
df['lower']=lowers
df['link']=links

In [66]:
df.to_csv("vector.csv")
df.head(5)

,feature,size,upper,lower,link
0,"[t.id, =, mc.movie_id, t.id, =, mi.movie_id, t...",4094363.0,"[1.0, 1.0, 0.6330935251798561, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,"[t.id, =, mi.movie_id, t.id, =, mi_idx.movie_i...",4085608.0,"[1.0, 0.0, 0.9568345323741008, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[t.id, =, mi.movie_id, t.id, =, mi_idx.movie_i...",138.0,"[1.0, 0.16666666666666666, 0.2302158273381295,...","[0.0, 0.0, 0.2302158273381295, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[mk.keyword_id, <, 72871]",4364160.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[t.id, =, ci.movie_id, t.kind_id, <, 7, t.prod...",1332632.0,"[1.0, 1.0, 0.7769784172661871, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [67]:
ls = []
for i in range(len(uppers)):
    up = uppers[i]
    lo = lowers[i]
    lk = links[i]
    tmp = np.append(up,lo)
    tmp = np.append(tmp,lk)
    ls.append(tmp)
ls

[array([1.        , 1.        , 0.63309353, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.12844037, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 array([1.        , 0.        , 0.95683453, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.

In [68]:
test_ls = []
for i in range(len(test_uppers)):
    up = test_uppers[i]
    lo = test_lowers[i]
    lk = test_links[i]
    tmp = np.append(up,lo)
    tmp = np.append(tmp,lk)
    test_ls.append(tmp)

In [69]:
datav2 = pd.DataFrame(np.array(ls))
datav2

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.0,1.000000,0.633094,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000000,0.956835,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.0,0.166667,0.230216,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.000000,0.776978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9996,1.0,1.000000,0.877698,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9997,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
test_data = pd.DataFrame(np.array(test_ls))
test_data

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.0,1.0,0.913669,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1,1.0,1.0,0.546763,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.942446,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,1.0,1.0,0.942446,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.000000,1.0,0.097342,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1996,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1997,1.0,1.0,1.000000,1.0,0.001328,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [71]:
labels = df['size']
labels_min = labels.min()
labels_max = labels.max()
#labels = (labels-labels.min())/(labels.max()-labels.min())
labels

0        4094363.0
1        4085608.0
2            138.0
3        4364160.0
4        1332632.0
           ...    
9995          39.0
9996    13829250.0
9997    26945073.0
9998     6800601.0
9999     2636013.0
Name: size, Length: 10000, dtype: float64

In [72]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [73]:
train = datav2.copy()
train_label = labels
len(train.keys())

60

In [74]:
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[len(train.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='mean_squared_logarithmic_error',
                optimizer=optimizer,
                metrics=['accuracy'])
    return model

In [75]:
model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               7808      
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 8)                 264       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 18,417
Trainable params: 18,417
Non-trainable params: 0
_________________________________________________________________


In [76]:
from sklearn.model_selection import train_test_split
datav2['label'] = labels
train_data, valid_data = train_test_split(datav2, test_size=0.2)
train_labels = train_data.pop('label')
valid_labels = valid_data.pop('label')

In [77]:
class PrintDot(keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 20


history = model.fit(
  train_data, train_labels,validation_data=[valid_data,valid_labels],batch_size = 32,
  epochs=EPOCHS)

Epoch 1/20
250/250 [==============================] - 1s 3ms/step - loss: 22.3747 - accuracy: 0.0108 - val_loss: 16.3109 - val_accuracy: 0.0105
Epoch 2/20
250/250 [==============================] - 0s 2ms/step - loss: 15.0338 - accuracy: 0.0109 - val_loss: 13.4054 - val_accuracy: 0.0105
Epoch 3/20
250/250 [==============================] - 0s 2ms/step - loss: 12.6512 - accuracy: 0.0109 - val_loss: 11.4682 - val_accuracy: 0.0105
Epoch 4/20
250/250 [==============================] - 0s 2ms/step - loss: 11.1636 - accuracy: 0.0109 - val_loss: 10.2807 - val_accuracy: 0.0105
Epoch 5/20
250/250 [==============================] - 0s 2ms/step - loss: 10.2524 - accuracy: 0.0109 - val_loss: 9.2849 - val_accuracy: 0.0105ETA: 0s - loss: 10.3183 - accuracy: 0.010
Epoch 6/20
250/250 [==============================] - 0s 2ms/step - loss: 9.6931 - accuracy: 0.0109 - val_loss: 8.7315 - val_accuracy: 0.0105
Epoch 7/20
250/250 [==============================] - 0s 2ms/step - loss: 9.0826 - accuracy: 0.010

In [78]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
15,5.822238,0.010875,5.595175,0.0105,15
16,5.704085,0.010875,5.353706,0.0105,16
17,5.542598,0.010875,5.637673,0.0105,17
18,5.284683,0.010875,5.324582,0.0105,18
19,5.232992,0.010875,4.968560,0.0105,19


In [79]:
loss = model.evaluate(train_data, train_labels)

loss

250/250 [==============================] - 1s 2ms/step - loss: 4.4268 - accuracy: 0.0109


[4.4268317222595215, 0.010874999687075615]

In [80]:
train_predictions = model.predict(train).flatten()
train_predictions

array([3.9012740e+06, 5.4033881e+05, 3.1902096e+01, ..., 1.0683973e+07,
       1.4491730e+06, 6.7975210e+06], dtype=float32)

In [81]:
test_predictions = model.predict(test_data).flatten()
test_predictions

array([ 43630.605,   4550.654, 207812.11 , ...,  58547.67 ,  42884.277,
        60716.598], dtype=float32)

In [82]:
index=[]
predict=[]
for i in range(len(test_predictions)):
    index.append(i)
    predict.append(abs(test_predictions[i]))

In [83]:
output = pd.DataFrame({'Query ID':index,'Predicted Cardinality':predict})
output.to_csv("output.csv",index = False)

In [84]:
output.head()

,Query ID,Predicted Cardinality
0,0,4.363061e+04
1,1,4.550654e+03
2,2,2.078121e+05
3,3,2.874524e+06
4,4,6.105740e+04


In [85]:
def load_data(file_name):
    joins = []
    predicates = []
    tables = []

    # Load queries
    with open(file_name + ".csv", 'rU') as f:
        data_raw = list(list(rec) for rec in csv.reader(f, delimiter='#'))
        for row in data_raw:
            tables.append(row[0].split(','))
            joins.append(row[1].split(','))
            predicates.append(row[2].split(','))
    print("Loaded queries")
    
    return joins, predicates, tables

In [86]:
train_joins,train_predicates,train_tables = load_data("train")
test_joins,test_predicates,test_tables = load_data("test_without_label")
test_joins

Loaded queries
Loaded queries


'U' mode is deprecated


[['t.id=mi_idx.movie_id', 't.id=mk.movie_id'],
 ['t.id=mi.movie_id', 't.id=mi_idx.movie_id'],
 ['t.id=mk.movie_id'],
 ['t.id=mc.movie_id'],
 ['t.id=mc.movie_id', 't.id=mi_idx.movie_id'],
 ['t.id=mc.movie_id', 't.id=mk.movie_id'],
 [''],
 ['t.id=mk.movie_id'],
 ['t.id=mc.movie_id', 't.id=mi.movie_id'],
 ['t.id=ci.movie_id', 't.id=mi_idx.movie_id'],
 ['t.id=mk.movie_id'],
 ['t.id=ci.movie_id'],
 ['t.id=mi.movie_id', 't.id=mk.movie_id'],
 ['t.id=mi.movie_id'],
 ['t.id=mc.movie_id'],
 [''],
 ['t.id=mk.movie_id'],
 ['t.id=ci.movie_id'],
 ['t.id=ci.movie_id'],
 ['t.id=mk.movie_id'],
 ['t.id=ci.movie_id', 't.id=mk.movie_id'],
 [''],
 ['t.id=ci.movie_id', 't.id=mi.movie_id'],
 [''],
 ['t.id=mi.movie_id'],
 [''],
 ['t.id=mi.movie_id', 't.id=mi_idx.movie_id'],
 [''],
 [''],
 ['t.id=mc.movie_id', 't.id=mi_idx.movie_id'],
 ['t.id=mi.movie_id'],
 ['t.id=mc.movie_id'],
 [''],
 [''],
 [''],
 ['t.id=mc.movie_id'],
 ['t.id=mi.movie_id', 't.id=mi_idx.movie_id'],
 ['t.id=mc.movie_id', 't.id=ci.movie_id']

In [87]:
jieba.load_userdict("dict.txt")
interpunctuations=['#','##',' ','title','t','movie_companies','mc','cast_info','ci','movie_info','mi','movie_info_idx','mi_idx','movie_keyword','mk']
table_name = ['title t','movie_companies mc','cast_info ci','movie_info mi','movie_info_idx mi_idx','movie_keyword mk']

In [88]:
joins_set = set()
for line in train_joins:
    for item in line:
        joins_set.add(item)
joins_set = list(joins_set)
joins_set

['',
 't.id=ci.movie_id',
 't.id=mi_idx.movie_id',
 't.id=mc.movie_id',
 't.id=mi.movie_id',
 't.id=mk.movie_id']

In [89]:
table_features = []
link_features = []
for line in range(len(train_joins)):
    table_vector = np.zeros(len(table_name))
    link_vector = np.zeros(len(joins_set))
    for item in train_tables[line]:
        id = table_name.index(item)
        table_vector[id] = 1
    for item in train_joins[line]:
        id = joins_set.index(item)
        link_vector[id] = 1
    table_features.append(table_vector)
    link_features.append(link_vector)

In [90]:
test_table_features = []
test_link_features = []
for line in range(len(test_joins)):
    table_vector = np.zeros(len(table_name))
    link_vector = np.zeros(len(joins_set))
    for item in test_tables[line]:
        id = table_name.index(item)
        table_vector[id] = 1
    for item in test_joins[line]:
        id = joins_set.index(item)
        link_vector[id] = 1
    test_table_features.append(table_vector)
    test_link_features.append(link_vector)

In [91]:
ls = []
for i in range(len(uppers)):
    up = uppers[i]
    lo = lowers[i]
    lk = links[i]
    tb = table_features[i]
    jo = link_features[i]
    tmp = np.append(up,lo)
    tmp = np.append(tmp,lk)
    tmp = np.append(tmp,tb)
    tmp = np.append(tmp,jo)
    ls.append(tmp)

In [92]:
test_ls = []
for i in range(len(test_uppers)):
    up = test_uppers[i]
    lo = test_lowers[i]
    lk = test_links[i]
    tb = test_table_features[i]
    jo = test_link_features[i]
    tmp = np.append(up,lo)
    tmp = np.append(tmp,lk)
    tmp = np.append(tmp,tb)
    tmp = np.append(tmp,jo)
    test_ls.append(tmp)

In [93]:
train2 = pd.DataFrame(np.array(ls))
train2

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.0,1.000000,0.633094,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.000000,0.956835,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.166667,0.230216,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.000000,0.776978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9996,1.0,1.000000,0.877698,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9997,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9998,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [94]:
test_data2 = pd.DataFrame(np.array(test_ls))
test_data2

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.0,1.0,0.913669,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,0.546763,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.942446,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,0.942446,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1.0,1.000000,1.0,0.097342,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1996,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1997,1.0,1.0,1.000000,1.0,0.001328,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1998,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [95]:
print(len(train2.keys()))
train2_label = labels
train2_label

72


0        4094363.0
1        4085608.0
2            138.0
3        4364160.0
4        1332632.0
           ...    
9995          39.0
9996    13829250.0
9997    26945073.0
9998     6800601.0
9999     2636013.0
Name: size, Length: 10000, dtype: float64

In [96]:
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[len(train2.keys())]),
        layers.Dense(256, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='mean_squared_logarithmic_error',
                optimizer=optimizer,
                metrics=['accuracy'])
    return model

In [97]:
model = build_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               9344      
                                                                 
 dense_16 (Dense)            (None, 256)               33024     
                                                                 
 dense_17 (Dense)            (None, 1024)              263168    
                                                                 
 dense_18 (Dense)            (None, 8)                 8200      
                                                                 
 dense_19 (Dense)            (None, 1)                 9         
                                                                 
Total params: 313,745
Trainable params: 313,745
Non-trainable params: 0
_________________________________________________________________


In [98]:
EPOCHS = 20

history = model.fit(
  train2, train2_label,batch_size = 32,
  epochs=EPOCHS, validation_split = 0.2, verbose=0)

In [99]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
15,5.649946,0.0105,5.091595,0.012,15
16,5.426371,0.0105,7.305457,0.012,16
17,5.357687,0.0105,4.982342,0.012,17
18,5.376669,0.0105,4.540005,0.012,18
19,5.239965,0.0105,4.601365,0.012,19


In [100]:
loss = model.evaluate(train2, train2_label, verbose=2)

loss

313/313 - 1s - loss: 4.3390 - accuracy: 0.0108 - 626ms/epoch - 2ms/step


[4.338954448699951, 0.01080000028014183]

In [101]:
train_predictions = model.predict(train2).flatten()
train_predictions

array([4.8168200e+06, 7.9360262e+05, 1.5220117e+02, ..., 2.7821868e+07,
       1.6826421e+06, 4.0940882e+06], dtype=float32)

In [102]:
test_predictions = model.predict(test_data2).flatten()
test_predictions

array([207737.62  ,   3033.4941, 166570.34  , ..., 188088.38  ,
        68518.6   ,  49605.938 ], dtype=float32)

In [103]:
index=[]
predict=[]
for i in range(len(test_predictions)):
    index.append(i)
    predict.append(abs(test_predictions[i]))

In [104]:
output = pd.DataFrame({'Query ID':index,'Predicted Cardinality':predict})
output.to_csv("output2.csv",index = False)
output.head()

,Query ID,Predicted Cardinality
0,0,2.077376e+05
1,1,3.033494e+03
2,2,1.665703e+05
3,3,3.291675e+06
4,4,1.458912e+05
